### Imports

In [1]:
### Standard imports
import os
import shutil
import numpy as np
import pandas as pd
from glob import glob
from os.path import join


### Custom helpers
import sys
sys.path.append("classes")
from Cloud import Cloud
from Button import Button
from DicomDir import DicomDir 
from helpers import *
from Subject import Subject 

# ### Load objects
cloud = Cloud()

### Save session JSON

In [2]:
def gen_json(json_path, project, subject, task, is_ascending, operator):
    import json
    info = dict()
    
    ### JSON information
    info["age"]          = subject.age
    info["sex"]          = subject.sex
    info["project"]      = project
    info["is_gng"]       = task
    info["is_ascending"] = is_ascending
    info["operator"]     = str(operator)
    
    ### Create directory (if needed)
    dst_dir = os.path.dirname(json_path)
    if not os.path.isdir(dst_dir):
        os.makedirs(dst_dir)
    
    ### Save JSON
    with open(json_path, "w") as f:
        json.dump(info, f)

### Run

In [3]:
projects = ["engage"]
tasks = ["gonogo", "conscious", "nonconscious"]
time_sessions = ["000_data_archive"]
output = "/Volumes/hd_4tb"

### Iterate over projects
for project in projects:
#         time_sessions = project_config["time_sessions"]
#         subjects = load_project_subjects(project)
    subjects = pd.read_csv("/Volumes/group/PANLab_Datasets/ENGAGE/ENGAGE_BV_sublist_n108_Final_List.csv")
    subjects = subjects.subNum

    ### Iterate over time_session/subject
    for time_session in time_sessions:
        for subject_id in subjects:
            print("Generating data for subject %s" % subject_id)

            ### Load subject specific data
            subject = Subject(subject_id)

            ### Load the button load files
            buttons = Button()
            buttons.find_subject_session_logs(project, subject_id, time_session)
            
            ### Skip subject if they have no onsets
            if len(buttons.onsets) == 0:
                print("No button logs for %s %s" % (subject_id, time_session))
                continue
                
            for i, task in enumerate(tasks):
                print("Generating data for task %s" % task)
                
                ### Get the correct onsets from the ones gathered
                onsets = buttons.onsets[i]
                
                ### Download task dicom from Flywheel
                dicom_path = cloud.download(project, subject_id, time_session, task)

                ### Load dicoms and cut the first 3 volumes
                fmri_data = DicomDir(dicom_path)
                fmri_data.cut_volumes(3)
                
                ### Generate JSON for patient - time_session - task
                json_path = join(output, "nifti", project, time_session, subject_id, task, "info.json")
                if not os.path.isfile(json_path):
                    gen_json(json_path, project, subject, task, fmri_data.is_ascending, fmri_data.operator)

                for num_volume in range(1, fmri_data.num_volumes - 2):
                    print("Generating data for volume %d" % num_volume)
                    
                    ### Generate the training data from all the features
                    df = gen_data(fmri_data, num_volume, onsets)
                    
                    filename  = "volume_%d.feather" % num_volume
                    save_path = join(output, "nifti", project, time_session, subject_id, task, filename)
                    df.to_feather(save_path)
                    del df
                    
                ### Delete dicom data
                shutil.rmtree(fmri_data.directory)

Training on subject PA30563
Training on task gonogo
unzipping /Users/pbezuhov/tmp/engage-PA30563-000_data_archive-gonogo.dicom.zip...


  0%|          | 7/6930 [00:00<02:10, 53.09it/s]

removing old path /Users/pbezuhov/tmp/engage-PA30563-000_data_archive-gonogo.dicom.zip...
loading dicoms from /Users/pbezuhov/tmp/engage-PA30563-000_data_archive-gonogo...
Sorting dicoms by trigger time...


100%|██████████| 6930/6930 [02:01<00:00, 56.99it/s]


Generating data for volume 1


100%|██████████| 1041/1041 [08:47<00:00,  1.97it/s]


Generating data for volume 2


 56%|█████▌    | 80042/143360 [26:46<23:56, 44.07it/s]  

KeyboardInterrupt: 